In [63]:
import pandas as pd
stats = pd.read_csv("combined_stats.csv")
stats = stats.drop(columns=['Unnamed: 0'])
stats.head()

,Player,year,depth,pass_attempts,complete_pass,incomplete_pass,passing_yards,passing_air_yards,pass_td,interception,...,vacated_reception_td,vacated_rush_attempts,vacated_rushing_yards,vacated_run_td,vacated_touches,vacated_total_yards,Votes,position_QB,position_RB,position_WR
0,A.J. Brown,2019,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,6.0,185.0,656.0,6.0,383.0,2420.0,0.0,0,0,1
1,A.J. Brown,2020,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,4.0,8.0,19.0,0.0,176.0,1338.0,0.0,0,0,1
2,A.J. Brown,2021,1.0,2.0,0.0,2.0,0.0,42.0,0.0,0.0,...,7.0,88.0,365.0,0.0,345.0,2147.0,0.0,0,0,1
3,A.J. Derby,2016,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,16.0,61.0,194.0,1.0,176.0,1318.0,0.0,0,0,0
4,A.J. Derby,2017,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,7.0,228.0,929.0,8.0,990.0,5994.0,0.0,0,0,0


In [64]:
null_series = pd.isnull(stats).sum().sort_values(ascending=False).head(20)
null_series = null_series[null_series > 0].index
stats = stats.drop(columns=null_series)

In [65]:
stats = stats.drop(list(stats[pd.isnull(stats['depth'])].index))

In [66]:
stats.head()

,Player,year,depth,pass_attempts,complete_pass,incomplete_pass,passing_yards,passing_air_yards,pass_td,interception,...,average_rec_ypg,average_rush_ypg,average_ypg,average_ppg,injuries,career_injuries,Votes,position_QB,position_RB,position_WR
0,A.J. Brown,2019,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,65.69,3.75,69.44,13.57,0.0,0.0,0.0,0,0,1
1,A.J. Brown,2020,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,70.87,2.00,72.87,15.45,3.0,3.0,0.0,0,0,1
2,A.J. Brown,2021,1.0,2.0,0.0,2.0,0.0,42.0,0.0,0.0,...,69.65,1.63,71.28,14.99,3.0,6.0,0.0,0,0,1
3,A.J. Derby,2016,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,26.67,0.00,26.67,4.83,2.0,2.0,0.0,0,0,0
4,A.J. Derby,2017,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,26.93,0.00,26.93,5.56,1.0,3.0,0.0,0,0,0


In [67]:
test = stats[stats['year'] == 2021]
train = stats[stats['year'] < 2021]

In [49]:
from sklearn.linear_model import Ridge

In [68]:
regression = Ridge(alpha=.1)

In [207]:
attributes = ['pass_td', 'total_tds', 'passing_yards', 'total_yards',
       'career_pass_td', 'complete_pass', 'career_total_tds',
       'passing_air_yards', 'yards_pct', 'fantasy_points_ppr', 'pass_attempts',
       'touches', 'career_passing_yards', 'career_complete_pass', 'ypg',
       'incomplete_pass', 'career_passing_air_yards', 'career_pass_attempts',
       'pass_ypg', 'win_pct', 'average_ypa']

In [211]:
regression.fit(train[attributes], train['Votes'])

Ridge(alpha=0.1)

In [212]:
predictions = regression.predict(test[attributes])
predictions = pd.DataFrame(predictions, columns=['predictions'], index=test.index)

In [213]:
combo = pd.concat([test[['Player', 'Votes']], predictions], axis = 1)
combo

,Player,Votes,predictions
2,A.J. Brown,0.0,0.078140
7,A.J. Dillon,0.0,0.294152
16,A.J. Green,0.0,-0.011897
32,Aaron Jones,0.0,0.148684
43,Aaron Rodgers,39.0,6.595673
...,...,...,...
4626,Zach Gentry,0.0,-0.074435
4638,Zach Pascal,0.0,-0.039489
4641,Zach Wilson,0.0,-4.576564
4648,Zack Moss,0.0,0.081947


In [214]:
combo.sort_values('predictions', ascending=False)

,Player,Votes,predictions
3263,Matthew Stafford,0.0,8.257118
43,Aaron Rodgers,39.0,6.595673
4310,Tom Brady,10.0,6.575553
2621,Justin Herbert,0.0,5.947842
1086,Dak Prescott,0.0,5.380850
...,...,...,...
2609,Justin Fields,0.0,-2.994848
3242,Matt Ryan,0.0,-3.065362
3979,Sam Darnold,0.0,-3.506434
4641,Zach Wilson,0.0,-4.576564


In [243]:
def error_metric(df):
    actual_ranking = df.sort_values('Votes', ascending=False).head(1)
    predicted = df.sort_values('predictions', ascending=False)
    ps = []
    found = 0
    seen = 1
    for index, row in predicted.iterrows():
        if row['Player'] in actual_ranking['Player'].values:
            found += 1
            ps.append(found/seen)
        seen += 1
    return sum(ps) / len(ps)

In [244]:
error_metric(combo)

0.5

In [245]:
stats['year'].sort_values(ascending=False)

4652    2021
2270    2021
2233    2021
490     2021
1060    2021
        ... 
1897    2012
2511    2012
4135    2012
3377    2012
3779    2012
Name: year, Length: 4600, dtype: int64

In [246]:
years = list(range(2012, 2022))
years

[2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021]

In [247]:
aps = []
all_predictions = []
for year in years[5:]:
    train = stats[stats['year'] < year]
    test = stats[stats['year'] == year]
    regression.fit(train[attributes], train['Votes'])
    predictions = regression.predict(test[attributes])
    predictions = pd.DataFrame(predictions, columns=['predictions'], index=test.index)
    combo = pd.concat([test[['Player', 'Votes']], predictions], axis = 1)
    all_predictions.append(combo)
    aps.append(error_metric(combo))
sum(aps) / len(aps)

0.72

In [248]:
def add_ranks(df):
    df = df.sort_values(['Votes', 'predictions'], ascending=False)
    df['rank'] = list(range(1, df.shape[0]+1))
    df = df.sort_values('predictions', ascending=False)
    df['predicted_rank'] = list(range(1, df.shape[0]+1))
    df['diff'] = df['predicted_rank'] - df['rank']
    return df

In [249]:
ranking = add_ranks(all_predictions[4])
ranking[ranking['rank'] < 4].sort_values('diff', ascending=False)

,Player,Votes,predictions,rank,predicted_rank,diff
1000,Cooper Kupp,1.0,0.126589,3,78,75
43,Aaron Rodgers,39.0,6.595673,1,2,1
4310,Tom Brady,10.0,6.575553,2,3,1


In [250]:
def backtest(stats, model, year, predictors):
    aps = []
    all_predictions = []
    for year in years[5:]:
        train = stats[stats['year'] < year]
        test = stats[stats['year'] == year]
        regression.fit(train[attributes], train['Votes'])
        predictions = regression.predict(test[attributes])
        predictions = pd.DataFrame(predictions, columns=['predictions'], index=test.index)
        combo = pd.concat([test[['Player', 'Votes']], predictions], axis = 1)
        combo = add_ranks(combo)
        all_predictions.append(combo)
        aps.append(error_metric(combo))
    return sum(aps) / len(aps), aps, pd.concat(all_predictions)

In [251]:
mean_ap, aps, all_predictions = backtest(stats, regression, years, attributes)

In [252]:
mean_ap

0.72

In [253]:
all_predictions[all_predictions['rank'] < 2].sort_values('diff', ascending=False)

,Player,Votes,predictions,rank,predicted_rank,diff
4306,Tom Brady,40.0,3.963480,1,10,9
43,Aaron Rodgers,39.0,6.595673,1,2,1
3575,Patrick Mahomes,41.0,11.139871,1,1,0
2868,Lamar Jackson,50.0,8.352491,1,1,0
42,Aaron Rodgers,44.0,11.022583,1,1,0
